In [1]:
import os
import sys
from pathlib import Path

In [2]:
current_file_path = os.path.abspath('')
root_file_path = str(Path(current_file_path).parent)
sys.path.append(root_file_path)

In [3]:
from deep_globe_seg.helpers import list_subfolders_and_files, CFG, visualize, round_clip_0_1, denormalize, Dataset

In [11]:
# Define custom parameters
custom_dataset_params = {
    'img_size': (512, 512),
    'input_shape': (512, 512, 3),
}

custom_hyper_params = {
    'batch_size': 16,
    'learning_rate': 0.0001,
    'num_classes': 1,
    'epochs': 40
}

# Initialize the CFG class with custom parameters
configs = CFG(image_folder=os.path.join(root_file_path, 'data'),
                  saved_model_folder=os.path.join(root_file_path, 'saved_models'),
                  tensorboard_logs_path=os.path.join(root_file_path, 'logs'),
                  model_type='unet',
                  hyper_params=custom_hyper_params,
                  dataset_params=custom_dataset_params)

In [15]:
Path(configs.Path.saved_model_folder).mkdir(parents = True, exist_ok = True)
Path(configs.Path.tensorboard_logs_path).mkdir(parents = True, exist_ok = True)

In [14]:
# Replace 'your_base_folder_path' with the path to your base folder
list_subfolders_and_files(configs.Path.image_folder)

Subfolder: /Users/sgpeytrignet/Documents/deepglobe-road-segmentation/data/valid
  100794_sat.jpg
  100905_sat.jpg
  102867_sat.jpg
  10417_sat.jpg
  106553_sat.jpg
  10678_sat.jpg
  107243_sat.jpg
  109245_sat.jpg
  11053_sat.jpg
  112600_sat.jpg
Subfolder: /Users/sgpeytrignet/Documents/deepglobe-road-segmentation/data/test
  100393_sat.jpg
  100995_sat.jpg
  101982_sat.jpg
  10206_sat.jpg
  102240_sat.jpg
  102765_sat.jpg
  104859_sat.jpg
  105342_sat.jpg
  105546_sat.jpg
  105695_sat.jpg
Subfolder: /Users/sgpeytrignet/Documents/deepglobe-road-segmentation/data/train


  100034_mask.png
  100034_sat.jpg
  100081_mask.png
  100081_sat.jpg
  100129_mask.png
  100129_sat.jpg
  100703_mask.png
  100703_sat.jpg
  100712_mask.png
  100712_sat.jpg


In [6]:
# Import CV packages
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import albumentations as A

# Import tensorflow, keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Import image segmentation package
import os
os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Read metadata and filter for training data
metadata_df = pd.read_csv(configs.Path.metadata)
metadata_df = metadata_df[metadata_df['split'] == 'train']

# Select relevant columns and update paths
metadata_df = metadata_df[['image_id', 'sat_image_path', 'mask_path']]
metadata_df['sat_image_path'] = metadata_df['sat_image_path'].apply(lambda img_pth: os.path.join(configs.Path.image_folder, img_pth))
metadata_df['mask_path'] = metadata_df['mask_path'].apply(lambda mask_pth: os.path.join(configs.Path.image_folder, mask_pth))

# Shuffle DataFrame
metadata_df = metadata_df.sample(frac=1, random_state=9).reset_index(drop=True)

# Split into train, validation, and test sets (80/10/10 split)
test_df = metadata_df.sample(frac=0.1, random_state=9)
remaining_df = metadata_df.drop(test_df.index)
valid_df = remaining_df.sample(frac=0.1, random_state=9)
train_df = remaining_df.drop(valid_df.index)

# Print informative message about dataset split
print("Dataset successfully loaded and split into training, validation, and test sets:")
print(f"Number of samples in training set: {len(train_df)}")
print(f"Number of samples in validation set: {len(valid_df)}")
print(f"Number of samples in test set: {len(test_df)}")

Dataset successfully loaded and split into training and validation sets:
Number of samples in training set: 5603
Number of samples in validation set: 623


In [17]:
train_df

,image_id,sat_image_path,mask_path
0,812075,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
1,690443,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
2,385863,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
3,374296,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
4,641221,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
...,...,...,...
6218,769666,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
6220,823500,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
6221,354791,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
6224,736666,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...


In [18]:
valid_df

,image_id,sat_image_path,mask_path
4932,563354,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
4117,701263,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
2008,178641,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
1930,371829,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
3795,238434,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
...,...,...,...
4987,872854,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
567,189336,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
5697,196693,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...
1381,30137,/Users/sgpeytrignet/Documents/deepglobe-road-s...,/Users/sgpeytrignet/Documents/deepglobe-road-s...


In [ ]:
test_df

In [19]:
class_dict = pd.read_csv(configs.Path.class_dict)
# Get class names
class_names = class_dict['name'].tolist()
# Get class RGB values
class_rgb_values = class_dict[['r','g','b']].values.tolist()

print('All dataset classes and their corresponding RGB values in labels:')
print('Class Names: ', class_names)
print('Class RGB values: ', class_rgb_values)

All dataset classes and their corresponding RGB values in labels:
Class Names:  ['road', 'background']
Class RGB values:  [[255, 255, 255], [0, 0, 0]]


In [ ]:
BACKBONE = 'resnet50'
BATCH_SIZE = cfg.HyperParameter.batch_size
CLASSES = ['road']
LR = cfg.HyperParameter.learning_rate
EPOCHS = cfg.HyperParameter.epochs

In [ ]:
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

In [ ]:
# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
total_loss = sm.losses.binary_focal_dice_loss

# Evaluation metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimizer, loss and metrics
model.compile(optim, total_loss, metrics)

In [ ]:
# Dataset for train images
train_dataset = Dataset(
    train_df,
    shape=cfg.Dataset.img_size,
    classes=CLASSES,
    preprocessing=get_preprocessing(preprocess_input),
    augmentation=get_training_augmentation()
)

# Dataset for validation images
valid_dataset = Dataset(
    valid_df,
    shape=cfg.Dataset.img_size,
    classes=CLASSES,
    preprocessing=get_preprocessing(preprocess_input)
)

train_dataloader = Dataloader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloader(valid_dataset, batch_size=1, shuffle=False)

In [ ]:
# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, *cfg.Dataset.img_size, 3)
assert train_dataloader[0][1].shape == (BATCH_SIZE, *cfg.Dataset.img_size, n_classes)

In [ ]:
# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint(cfg.Path.model_save_path, save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  patience=3,
                                  verbose=1, 
                                  mode='min',
                                  restore_best_weights=True)
]

In [ ]:
# train model
if os.path.isfile(cfg.Path.model_path):
    print("INFO ===========Running the Partially Trained Model===============")
    model.load_weights(cfg.Path.model_path)
    history = model.fit_generator(
        train_dataloader, 
        steps_per_epoch=len(train_dataloader), 
        epochs=EPOCHS, 
        callbacks=callbacks, 
        validation_data=valid_dataloader, 
        validation_steps=len(valid_dataloader),
    )
else:
    print("INFO ===========Running the Training of Model from Scratch===============")
    history = model.fit_generator(
        train_dataloader, 
        steps_per_epoch=len(train_dataloader), 
        epochs=EPOCHS, 
        callbacks=callbacks, 
        validation_data=valid_dataloader, 
        validation_steps=len(valid_dataloader),
    )

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
# plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

# Plot training & validation loss values
# plt.subplot(122)
plt.figure(figsize=(30, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

plt.figure(figsize=(30, 5))
plt.plot(history.history['f1-score'])
plt.plot(history.history['val_f1-score'])
plt.title('Model loss')
plt.ylabel('f1-score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
# load best weights
model.load_weights(cfg.Path.model_save_path) 

In [ ]:
scores = model.evaluate_generator(train_dataloader) # test_dataloader

print("Training loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean training {}: {:.5}".format(metric.__name__, value))

In [ ]:
scores = model.evaluate_generator(valid_dataloader) # test_dataloader

print("Validation loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean validation {}: {:.5}".format(metric.__name__, value))


In [ ]:
n = 3
# test_dataset
ids = np.random.choice(np.arange(len(valid_dataset)), size=n)

for i in ids:
    
    image, gt_mask = valid_dataset[i]  # test_dataset
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )